In [ ]:
example = "2333133121414131402"

In [ ]:
def generate_disk_blocks(disk_map):
    res = []
    for i, digit in enumerate(disk_map):
        if i % 2 == 0:
            res += [str(i // 2)] * int(digit)
        else:
            res += ["."] * int(digit)
    return res

In [ ]:
def compress_disk_blocks(disk_blocks):
    while "." in disk_blocks:
        disk_blocks[disk_blocks.index(".")] = disk_blocks[-1]
        disk_blocks = disk_blocks[:-1]
    return disk_blocks

In [ ]:
def calc_checksum(disk_blocks):
    assert "." not in disk_blocks
    return sum(i*int(block) for i, block in enumerate(disk_blocks))

In [ ]:
def calc(disk_map):
    disk_blocks = generate_disk_blocks(disk_map)
    disk_blocks = compress_disk_blocks(disk_blocks)
    return calc_checksum(disk_blocks)

In [ ]:
calc(example)

In [ ]:
from pathlib import Path

disk_map = Path("1.txt").read_text().strip()

In [ ]:
calc(disk_map)

In [ ]:
calc(disk_map)

In [ ]:
from dataclasses import dataclass
from enum import Enum

class BlockType(Enum):
    FREE = "free"
    FILE = "file"

@dataclass
class Block:
    block_type: BlockType
    size: int
    identifier: int | None = None

In [ ]:
def disk_map_to_blocks(disk_map) -> list[Block]:
    res = []
    for i, digit in enumerate(disk_map):
        if i % 2 == 0:
            res.append(Block(block_type=BlockType.FILE, size=int(digit), identifier=i // 2))
        else:
            res.append(Block(block_type=BlockType.FREE, size=int(digit)))
    return res


def print_blocks(blocks: list[Block]):
    for block in blocks:
        if block.block_type == BlockType.FREE:
            print("."*block.size, end="")
        else:
            print(str(block.identifier)*block.size, end="")
    print()


def correct_free_blocks(blocks: list[Block]) -> list[Block]:
    new_blocks = []
    acc_size = 0
    for i, block in enumerate(blocks):
        if block.block_type == BlockType.FREE:
            if not block.size:
                continue
            if i+1 < len(blocks) and blocks[i+1].block_type == BlockType.FREE:
                acc_size += block.size
                continue
            new_blocks.append(block)
            block.size += acc_size
            acc_size = 0
        else:
            new_blocks.append(block)
    return new_blocks


def compress_disk_map(disk_map, debug) -> list[Block]:
    blocks = disk_map_to_blocks(disk_map)
    if debug:
        print_blocks(blocks)
    blocks_done = []
    file_blocks = sum(1 for block in blocks if block.block_type == BlockType.FILE)
    while len(blocks_done) < file_blocks:
        i_last_file_block = next((len(blocks)-i-1 for i, block in enumerate(blocks[::-1]) if block.block_type == BlockType.FILE and block.identifier not in blocks_done))
        i_first_free_block = next((i for i, block in enumerate(blocks) if block.block_type == BlockType.FREE and blocks[i_last_file_block].size <= block.size and i < i_last_file_block), None)
        blocks_done.append(blocks[i_last_file_block].identifier)
        if i_first_free_block is None:
            continue
        blocks = (
            blocks[:i_first_free_block] + 
            [blocks[i_last_file_block], Block(BlockType.FREE, size=blocks[i_first_free_block].size - blocks[i_last_file_block].size)] + 
            blocks[i_first_free_block+1:i_last_file_block] + 
            [Block(block_type=BlockType.FREE, size=blocks[i_last_file_block].size)] + 
            blocks[i_last_file_block+1:]
        )
        blocks = correct_free_blocks(blocks)
        if debug:
            print_blocks(blocks)
    return blocks

In [ ]:
def calc2(disk_map, debug=False):
    blocks = compress_disk_map(disk_map, debug)
    i = 0
    s = 0
    for block in blocks:
        if block.block_type == BlockType.FILE:
            s += ((i + i + block.size - 1) * block.size) // 2 * block.identifier
        i += block.size
    return s

In [ ]:
calc2(example)

In [ ]:
calc2(disk_map)